# Preprocessing Tahap 2 (Feature Selection s.d. Target Engineering)

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("Data Gabungan dari Linkedin dan Jobstreet (3 Profesi).csv")
data.tail()

In [ ]:
data.info()

In [ ]:
def summarize_features(df):
    # first column will be data types of each feature
    summary = pd.DataFrame(df.dtypes, columns = ['dtypes'])
    summary = summary.reset_index()
    # how many missing values in each feature
    summary['Missing'] = df.isnull().sum().values
    # how many unique values in each feature (cardinality indicator)
    summary['Uniques'] = df.nunique().values

    return summary

summarize_features(data)

In [ ]:
len(data)

In [ ]:
data.drop(["company", "gaji", "sumber"], axis = 1, inplace = True)

In [ ]:
data.columns

In [ ]:
# subset -> Define in which columns to look for missing values

data.dropna(subset = ["median_gaji"], inplace = True)

In [ ]:
len(data)

In [ ]:
data.groupby(["job_name"]).size()

In [ ]:
data_awal = data.copy()

In [ ]:
len(data_awal) # "data_awal" yang disimpan aja, "data" dipakai di bawah2nya

In [ ]:
data_awal.rename(columns = {"job_name": "Nama Profesi Data",
                       "lokasi": "Lokasi Perusahaan", 
                       "tingkat_job": "Tingkat Pekerjaan", 
                       "lama_pengalaman": "Lama Pengalaman",
                       "jenis_job": "Jenis Pekerjaan", 
                       "ukuran_company": "Ukuran Perusahaan", 
                       "industri": "Industri", 
                       "median_gaji": "Gaji"}, inplace=True)

In [ ]:
#!pip install dython

In [ ]:
# KORELASI
from dython import nominal
nominal.associations(data_awal, 
                     nominal_columns = ["Nama Profesi Data", "Lokasi Perusahaan", "Tingkat Pekerjaan", "Jenis Pekerjaan", "Ukuran Perusahaan", "Industri"], 
                     numerical_columns = ["Lama Pengalaman", "Gaji"],
                     figsize = (13, 12), 
                     title = "Korelasi Antar Variabel",
                     filename = "Hasil Korelasi Dython")

# Coba ambil yang korelasinya >25% dan yang masuk akal
# Coba variabel "jenis_job" dihapus saja

In [ ]:
data.drop(["jenis_job"], axis = 1, inplace = True)

In [ ]:
summarize_features(data)

# Handling Missing Value

### 1. Menangani Missing Value di "ukuran_company"

In [ ]:
# Coba berdasarkan variabel -> "job_name"

import statistics

count_ukuran_company1 = data.groupby(["job_name"])["ukuran_company"].count()
mode_ukuran_company1 = data[data["ukuran_company"].notna()].groupby(["job_name"])["ukuran_company"].apply(statistics.mode)
len(mode_ukuran_company1)

In [ ]:
mode_ukuran_company1

In [ ]:
mode_ukuran_company1.values

In [ ]:
dict_group1a = {}

for indeks in mode_ukuran_company1.index:    
    group1a = {indeks: mode_ukuran_company1[indeks]}
    
    dict_group1a.update(group1a)

In [ ]:
missing_ukuran_company1 = data[pd.isna(data["ukuran_company"])]

for i in missing_ukuran_company1.index:
    for j in dict_group1a.keys():
        data_asli = (data["job_name"][i])
        
        if data_asli == j:
            data["ukuran_company"][i] = dict_group1a[j]

In [ ]:
pd.isna(data).sum()

### 2. Menangani Missing Value di "industri"

In [ ]:
# Coba berdasarkan variabel -> "job_name"

count_industri1 = data.groupby(["job_name"])["industri"].count() # Hasilnya adalah banyaknya data yang tidak missing
mode_industri1 = data[data["industri"].notna()].groupby(["job_name"])["industri"].apply(statistics.mode) # Pakainya modus karena data kategorik
len(mode_industri1)

In [ ]:
mode_industri1

In [ ]:
mode_industri1.values

In [ ]:
dict_group2a = {}

for indeks in mode_industri1.index:
    group2a = {indeks: mode_industri1[indeks]}

    dict_group2a.update(group2a)

In [ ]:
dict_group2a.values() # Sudah tidak ada yang nan

In [ ]:
missing_industri1 = data[pd.isna(data["industri"])]

for i in missing_industri1.index:
    for j in dict_group2a.keys():
        data_asli = (data["job_name"][i])
        
        if data_asli == j:
            data["industri"][i] = dict_group2a[j]

In [ ]:
pd.isna(data).sum()

### 3. Menangani Missing Value di "lama_pengalaman"

In [ ]:
# Coba berdasarkan variabel -> "job_name"

count_lama_pengalaman1 = data.groupby(["job_name"])["lama_pengalaman"].count() # Hasilnya adalah banyaknya data yang tidak missing
median_lama_pengalaman1 = data[data["lama_pengalaman"].notna()].groupby(["job_name"])["lama_pengalaman"].median() # Pakainya median karena data numerik

len(median_lama_pengalaman1)

In [ ]:
median_lama_pengalaman1

In [ ]:
median_lama_pengalaman1.values

In [ ]:
dict_group4a = {}

for indeks in median_lama_pengalaman1.index:      
    group4a = {indeks: median_lama_pengalaman1[indeks]}

    dict_group4a.update(group4a)

In [ ]:
missing_lama_pengalaman1 = data[pd.isna(data["lama_pengalaman"])]

for i in missing_lama_pengalaman1.index:
    for j in dict_group4a.keys():
        data_asli = (data["job_name"][i])
        
        if data_asli == j:
            data["lama_pengalaman"][i] = dict_group4a[j]

In [ ]:
pd.isna(data).sum() 

In [ ]:
data.groupby(["job_name"]).size()

# Handling Categorical Data

In [ ]:
data_oke = data.copy()
data_oke["job_name"].value_counts()

In [ ]:
summarize_features(data_oke)

In [ ]:
data_oke.info()

In [ ]:
# NOMINAL -> "job_name", "lokasi", "industri" (nanti pakai get_dummies)
# ORDINAL -> "tingkat_job", "ukuran_company" (nanti pakai OrdinalEncoder)

In [ ]:
# NOMINAL

# "job_name", "lokasi", "industri"
nominal_cols = data_oke[["job_name", "lokasi", "industri"]]
encoded_nominal = pd.get_dummies(data = nominal_cols)

# Gabungkan ke data asli
data_oke = pd.concat(objs = [encoded_nominal, data_oke], axis = 1)
data_oke.drop(nominal_cols, axis = 1, inplace = True)
data_oke.head(2)

In [ ]:
#!pip install category_encoders

In [ ]:
# ORDINAL
import category_encoders as ce

# 1. "tingkat_job"
encoder_tingkat_job = ce.OrdinalEncoder(cols = ["tingkat_job"], return_df = True, 
                                        mapping = [{"col": "tingkat_job", 
                                                    "mapping": {"magang": 0, "tingkat pemula": 1, "asosiasi": 2, 
                                                                "senior tingkat menengah": 3, "direktur": 4, "eksekutif": 5}}])

data_oke["tingkat_job"] = encoder_tingkat_job.fit_transform(data_oke["tingkat_job"])

# 2. "ukuran_company"
encoder_ukuran_company = ce.OrdinalEncoder(cols = ["ukuran_company"], return_df = True, 
                                           mapping = [{"col": "ukuran_company", 
                                                       "mapping": {"1-50 pekerja": 0, "51-200 pekerja": 1, "201-500 pekerja": 2,
                                                                   "501-1.000 pekerja": 3, "1.001-5.000 pekerja": 4, ">5.000 pekerja": 5}}])

data_oke["ukuran_company"] = encoder_ukuran_company.fit_transform(data_oke["ukuran_company"])

In [ ]:
data_oke.head(2)

# Target Engineering

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
len(data_awal)

In [ ]:
# Data Asli
sns.distplot(data_oke["median_gaji"], hist = False, kde = True, axlabel = "Gaji")

plt.savefig("gambar/9. Distplot Gaji (78 Data).png")

plt.show()

In [ ]:
# Data Hasil Transformasi

#perform Box-Cox transformation on original data
transformed_data, best_lambda = stats.boxcox(data_oke["median_gaji"]) 

#plot the distribution of the transformed data values
sns.distplot(transformed_data, hist = False, kde = True, axlabel = "Gaji Hasil Transformasi")

plt.savefig("gambar/10. Distplot Gaji Setelah Transformasi (78 Data).png")

plt.show()

In [ ]:
nilai_lambda = best_lambda
nilai_lambda

In [ ]:
print("Data Asli:", list(data_oke["median_gaji"][0:5]))
print("Data Hasil Transformasi:", transformed_data[0:5])

In [ ]:
(20000000.0**nilai_lambda - 1) / nilai_lambda

In [ ]:
# Transformasikan
data_oke["median_gaji"] = transformed_data

In [ ]:
data_oke["median_gaji"][0:5]

In [ ]:
data_oke.head()

In [ ]:
#data_oke.to_csv("Data Gabungan Hasil Preprocessing Tahap 2 (3 Profesi).csv", index = False)